In [49]:
import html2text

path = "/Users/timoblattner/Desktop/CH_BGer_001_1A-1-2000_2000-05-08.html"

h = html2text.HTML2Text()
h.ignore_links = True

with open(path, "r") as file:
    text = h.handle(file.read())
    print(text)



[AZA 3]

1A.1/2000/hzg

1P.1/2000



_           I. OEFFENTLICHRECHTLICHE ABTEILUNG_  

          ********************************** 



_8\. Mai 2000_  



Es wirken mit: Bundesrichter Aemisegger, Präsident

der I. öffentlichrechtlichen Abteilung, Bundesrichter

Aeschlimann, Bundesrichter Favre und Gerichtsschreiberin

Camprubi.



\---------



In Sachen



_August J u l e n_ , des Severin, Chalet Hermitage, Zermatt,  

Beschwerdeführer, vertreten durch Rechtsanwalt Dr. Thomas

Julen, Haus Aurora, Zermatt,



gegen



_Munizipalgemeinde Z e r m a t t_ ,  

_Kantonale Baukommission des Kantons W a l l i s_ ,  

_Staatsrat des Kantons W a l l i s_ ,  

_Kantonsgericht des Kantons W a l l i s_ , Öffentlich-  

rechtliche Abteilung,



betreffend

          Bauabschlag und Abbruchbefehl 

         (rechtliches Gehör, Ausnahmebewilligung), 

hat sich ergeben:



_A.-_  

August Julen führt in Findeln oberhalb von  

Zermatt in der Landwirtschaftszone ein Bergrestaurant.

Unweit davon in

In [152]:
from bs4 import BeautifulSoup

path = "/Users/timoblattner/Desktop/134 I 293.html"

file = open(path, "r")

html = BeautifulSoup(file.read())

# the content of the page
content = html.find("div", {"id": "content"})

# we want the Erwägung
erw = content.find('h1', text="Erwägungen")


# get every article


# we take the paragraphs
while erw:
    if erw.find('strong') is not None:
        index = erw.find('strong').text
        
        links = erw.findChildren("a", {"class": "LexLink"})
        for link in links:
            print(link)
            #print(link.findChildren("a" , recursive=False))
            link.extract()
        
        thinsp = erw.findChildren("span", {"class": "thinsp"})
        for t in thinsp:
            t.extract()
            
        notes = erw.findChildren("span", {"class": "note"})
        for note in notes:
            note.extract()
        
        pagebk = erw.findChildren("span", {"class": "pagebreak"})
        for bk in pagebk:
            bk.extract()
            
        #print(links)
        #print(erw.text)
        
        print("_____")

    erw = erw.findNext("p")

_____
_____
<a class="LexLink" href="/BV_49">Art.<span class="thinsp"> </span>49 Abs.<span class="thinsp"> </span>1 BV</a>
<a class="LexLink" href="/SchKG_92">Art.<span class="thinsp"> </span>92 Abs.<span class="thinsp"> </span>1 Ziff.<span class="thinsp"> </span>1a SchKG</a>
<a class="LexLink" href="/ZGB_896">Art.<span class="thinsp"> </span>896 ZGB</a>
<a class="LexLink" href="/ZGB_641a">Art.<span class="thinsp"> </span>641a ZGB</a>
<a class="LexLink" href="/SchKG_92">Art.<span class="thinsp"> </span>92 Abs.<span class="thinsp"> </span>1 Ziff.<span class="thinsp"> </span>1a SchKG</a>
<a class="LexLink" href="/ZGB_896">Art.<span class="thinsp"> </span>896 ZGB</a>
_____
<a class="LexLink" href="/SchKG_38">Art.<span class="thinsp"> </span>38 SchKG</a>
<a class="LexLink" href="/SchKG_44">Art.<span class="thinsp"> </span>44 SchKG</a>
<a class="LexLink" href="/BV_7">Art.<span class="thinsp"> </span>7 BV</a>
<a class="LexLink" href="/BV_10">Art.<span class="thinsp"> </span>10 Abs.<span clas